# IMPORTS

In [1]:
import pandas as pd
import numpy as np
from collections import OrderedDict
import math

# USER INPUTS

In [6]:
# Enter 1 if providing train and test dataset and 2 if providing only train dataset
files = int(input("Will you be entering:\n(a)Only Train dataset (1) or\n(b)Train and Test dataset (2) "))
train_path = ""
test_path = ""
filePath = ""
if(files == 1):
    filePath = input("Please enter file name: ")
elif(files == 2):
    train_path = input("Please enter train data file name: ")
    test_path = input("Please enter test data file name: ")


knn = int(input("Enter the Number of Neighbors (K) for KNN: "))
fold_count = 1; normalize = "n";
if(files == 1):
    normalize = "y" #input("Would you like to normalize the dataset (Y/N)? -> ").lower()
    kfold = "y" #input("Would you like to implement Cross Validation (Y/N)? -> ").lower()
    if(kfold == "y"):
        fold_count = 10 #int(input("Please enter the number of fold counts: "))
    elif(kfold == "n"):
        fold_count = 1
    else:
        print("This value should be an integer")

Will you be entering:
(a)Only Train dataset (1) or
(b)Train and Test dataset (2) 1
Please enter file name: data1.txt
Enter the Number of Neighbors (K) for KNN: 15


# FUNCTIONS

In [3]:
def featurecheck(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def get_fold_range(row_count,fold_count):
    k_main = []
    if(fold_count==1):
        step = round(0.23*row_count)
        temp = []
        temp.append(0)
        temp.append(step)
        k_main.append(temp)
    else:
        step = math.floor(row_count / fold_count)
        rem = row_count - fold_count * step
        p = 0
        
        for i in range(fold_count):
            val_add = 0
            if(rem > 0):
                val_add = 1
            temp = []
            temp.append(p)
            p = p + step + val_add
            temp.append(p)
            k_main.append(temp)
            rem -= 1
        #print(k_main)
    return(k_main)

def performance_calculation(prediction,actual):
    a = 0; b = 0; c = 0; d = 0;
    for i in range(len(prediction)):
        if((prediction[i] == 1) and (actual[i] == 1)):
            a += 1
        elif((prediction[i] == 0) and (actual[i] == 1)):
            b += 1
        elif((prediction[i] == 1) and (actual[i] == 0)):
            c += 1
        elif((prediction[i] == 0) and (actual[i] == 0)):
            d += 1
    return(a,b,c,d)

def normalizing_dataset(dataset):
    df11 = pd.DataFrame(dataset)
    #data_stats = [df11.mean().tolist(), df11.std().tolist()]
    data_stats = [df11.min().tolist(), df11.max().tolist()]
    #pd.DataFrame(data_stats)
    for z6 in range(len(dataset)):
        for z7 in range(len(dataset[z6])):
            dataset[z6][z7] = (dataset[z6][z7] - data_stats[0][z7])/(data_stats[1][z7]-data_stats[0][z7])
            #dataset[z6][z7] = (dataset[z6][z7] - data_stats[0][z7])/(data_stats[1][z7])
    return dataset

def check_sep(line):
    flag = ""
    if(" " in line):
        flag = " "
    elif("\t" in line):
        flag = "\t"
    else:
        print("Data should be sperated by \t or single space!")
    return flag

def dist_calc(c):
    sum1 = 0
    for s in c:
        sum1 += (s**2)
    sum1 = sum1**(1/2)
    return sum1

# MAIN KNN CODE
def main_knn(files, k_main, dataset, unique_labels, knn, df):
    accuracy_list = []; precision_list = []; recall_list = []; fmeasure_list = [];

    for i in k_main:
        train_data = {}; test_data = {}; pred_labels = {}; dist = {}; label_count = {};

        for j in range(row_count):
            if(j in range(i[0],i[1])):
                test_data[j] = dataset[j]
                pred_labels[j] = -999
            else:
                train_data[j] = dataset[j]

        for j2 in test_data.keys():
            d1 = test_data[j2]

            for l1 in unique_labels:
                label_count[l1] = 0

            for k in train_data.keys():
                d2 = train_data[k]
                c = np.array(d2) - np.array(d1)
                dist[k] = round(dist_calc(c),4)

            dd = dict(OrderedDict(sorted(dist.items(), key=lambda x: x[1]))) #
            k_vals = (list(dd.keys()))[0:knn]
            #print(k_vals)
            for sn1 in range(knn):
                temp_label = true_label[k_vals[sn1]]
                #print(temp_label)
                for l2 in unique_labels:
                    if(temp_label==l2):
                        label_count[l2] = label_count[l2] + 1
            #print(label_count)
            lab1 = -999; lab2 = -999;
            for (z8,z9) in label_count.items():
                if(z9>lab2):
                    lab1 = z8
                    lab2 = z9
            #print(str(lab1)+" --- "+str(lab2))
            pred_labels[j2] = lab1

        comp = list(pred_labels.values())
        fin = list(df.iloc[i[0]:i[1],df.shape[1]-1].values)
        (a,b,c,d) = performance_calculation(comp,fin)
        accuracy = (a+d)/(a+b+c+d)
        precision = a/(a+c)
        recall = a/(a+b)
        fmeasure = (2*a)/((2*a)+b+c)
        accuracy_list.append(accuracy); precision_list.append(precision);
        recall_list.append(recall); fmeasure_list.append(fmeasure);
        
    return(accuracy_list, precision_list, recall_list, fmeasure_list)

def display_measures(acc, prec, rec, fm):
    if(len(acc)==1):
        print("Algorithm Measures")
        print("Accuracy -> "+str(acc[0]))
        print("Precision -> "+str(prec[0]))
        print("Recall -> "+str(rec[0]))
        print("F-Measure -> "+str(fm[0]))
    else:
        avg_accuracy = 0; avg_precision = 0; avg_recall = 0; avg_fmeasure = 0;
        for i in range(len(acc)):
            print("----------------------------------------")
            print("Cross Validation Iteration -> "+str(i+1))
            print("Algorithm Measures")
            print("Accuracy -> "+str(acc[i]))
            print("Precision -> "+str(prec[i]))
            print("Recall -> "+str(rec[i]))
            print("F-Measure -> "+str(fm[i]))

            avg_accuracy += acc[i]; avg_precision += prec[i]; avg_recall += rec[i]; avg_fmeasure += fm[i];

        print("----------------------------------------")
        print("Average Measures")
        print("Average Accuracy -> "+str(avg_accuracy/len(acc)))
        print("Average Precision -> "+str(avg_precision/len(prec)))
        print("Average Recall -> "+str(avg_recall/len(rec)))
        print("Average F-Measure -> "+str(avg_fmeasure/len(fm)))

# DATA PRE-PROCESSING

In [4]:
file1 = ""; df1 = ""; file2 = ""; df2 = ""; df = "";
row_count = 0; dataset = []; full_dataset = []; 
k_main = []; true_label = {}; update_data = {}; unique_labels = []; 

if(files==2):
    file1 = open(train_path, "r")
    file2 = open(test_path, "r")
    lines = file1.readlines()
    lines2 = file2.readlines()
    flag11 = 2; flag12 = 2; #Line Seperator
    flag21 = flag22 = 2; #Chosing method to handle nominal values
    z1 = 0; temp_upd_count = -1;

    for line in lines:
        flag11 = check_sep(line)
        break
    for line in lines2:
        flag12 = check_sep(line)
        break

    for line in lines:
        data = line.strip().split(flag11)
        if(flag21 == 1):
            for i in data:
                if(featurecheck(i)==False):
                    data.remove(i)
        elif(flag21 == 2):
            for i in range(len(data)):
                if(featurecheck(data[i])==False):
                    if(data[i] in update_data.keys()):
                        data[i] = update_data[data[i]]
                    else:
                        temp_upd_count += 1
                        data[i] = temp_upd_count

        data = [float(ele) for ele in data]
        true_label[z1] = data[-1]
        z1 += 1
        dataset.append(data[:-1])
        full_dataset.append(data[:])

    for line in lines2:
        data = line.strip().split(flag12)
        if(flag22 == 1):
            for i in data:
                if(featurecheck(i)==False):
                    data.remove(i)
        elif(flag22 == 2):
            for i in range(len(data)):
                if(featurecheck(data[i])==False):
                    if(data[i] in update_data.keys()):
                        data[i] = update_data[data[i]]
                    else:
                        temp_upd_count += 1
                        data[i] = temp_upd_count

        data = [float(ele) for ele in data]
        true_label[z1] = data[-1]
        z1 += 1
        dataset.append(data[:-1])
        full_dataset.append(data[:])

    df1 = pd.read_csv(train_path, sep = flag11, lineterminator = '\n', header = None)
    df2 = pd.read_csv(test_path, sep = flag11, lineterminator = '\n', header = None)
    unique_labels = list(set(true_label.values()))
    df = pd.DataFrame(full_dataset)
    
    row_count = df.shape[0]
    k_main.append([df1.shape[0],len(dataset)])
    #(unique_labels,b) = np.unique(df1.iloc[:,df1.shape[1]].values,return_counts = True)
else:
    file = open(filePath, "r")
    lines = file.readlines()
    flag1 = 2 #Line Seperator
    flag2 = 2 #Chosing method to handle nominal values
    z1 = 0; temp_upd_count = -1;

    for line in lines:
        if(" " in line):
            flag1 = " "
        elif("\t" in line):
            flag1 = "\t"
        else:
            print("Data should be sperated by \t or single space!")
        break

    for line in lines:
        data = line.strip().split(flag1)
        if(flag2 == 1):
            for i in data:
                if(featurecheck(i)==False):
                    data.remove(i)
        elif(flag2 == 2):
            for i in range(len(data)):
                if(featurecheck(data[i])==False):
                    if(data[i] in update_data.keys()):
                        data[i] = update_data[data[i]]
                    else:
                        temp_upd_count += 1
                        data[i] = temp_upd_count
        data = [float(ele) for ele in data]
        true_label[z1] = data[-1]
        z1 += 1
        dataset.append(data[:-1])

    df = pd.read_csv(filePath, sep = flag1, lineterminator = '\n', header = None)
    if(normalize == "y"):
        dataset = normalizing_dataset(dataset)
    row_count = df.shape[0]
    col_count = df.shape[1]-1
    k_main = get_fold_range(row_count,fold_count)
    unique_labels = list(set(true_label.values()))
    #(unique_labels,b) = np.unique(df1.iloc[:,col_count+1].values,return_counts = True)

# Implementation and Results

In [5]:
(acc, prec, rec, fm) = main_knn(files, k_main, dataset, unique_labels, knn, df)

display_measures(acc, prec, rec, fm)

----------------------------------------
Cross Validation Iteration -> 1
Algorithm Measures
Accuracy -> 0.9649122807017544
Precision -> 1.0
Recall -> 0.9130434782608695
F-Measure -> 0.9545454545454546
----------------------------------------
Cross Validation Iteration -> 2
Algorithm Measures
Accuracy -> 0.9473684210526315
Precision -> 0.9375
Recall -> 0.8823529411764706
F-Measure -> 0.9090909090909091
----------------------------------------
Cross Validation Iteration -> 3
Algorithm Measures
Accuracy -> 0.9649122807017544
Precision -> 1.0
Recall -> 0.8571428571428571
F-Measure -> 0.9230769230769231
----------------------------------------
Cross Validation Iteration -> 4
Algorithm Measures
Accuracy -> 0.9122807017543859
Precision -> 1.0
Recall -> 0.7619047619047619
F-Measure -> 0.8648648648648649
----------------------------------------
Cross Validation Iteration -> 5
Algorithm Measures
Accuracy -> 0.9298245614035088
Precision -> 1.0
Recall -> 0.8
F-Measure -> 0.8888888888888888
-------